In [9]:
from PIL import Image
import numpy as np
from scipy.ndimage import convolve

def get_neighbors(row, col, rows, cols):
    # 4-way connectivity
    neighbors = []
    if row > 0: neighbors.append((row - 1, col))
    if row < rows - 1: neighbors.append((row + 1, col))
    if col > 0: neighbors.append((row, col - 1))
    if col < cols - 1: neighbors.append((row, col + 1))
    return neighbors

def calculate_weight(pixel1, pixel2):
    # Simple difference in intensity
    return abs(int(pixel1) - int(pixel2))


def sobel_filters(image):
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    # Convolution
    Gx = convolve(image, sobel_x)
    Gy = convolve(image, sobel_y)

    # Gradient magnitude
    G = np.sqrt(Gx**2 + Gy**2)

    return Gx, Gy, G

def construct_graph_with_gradients(image, G):
    """
    Construct an undirected weighted graph using the gradient magnitudes.

    Args:
    - image (np.array): Grayscale image.
    - G (np.array): Gradient magnitude for each pixel.

    Returns:
    - dict: Graph represented as a dictionary.
    """
    rows, cols = image.shape
    graph = {}

    for row in range(rows):
        for col in range(cols):
            graph[(row, col)] = {}
            for neighbor in get_neighbors(row, col, rows, cols):
                # Use inverse of gradient magnitude as weight
                weight = 1 / (G[neighbor] + 1e-5)  # Adding a small constant to avoid division by zero
                graph[(row, col)][neighbor] = weight
    
    return graph



In [13]:
# First, let's implement the functions we will use to calculate the shortest path and draw it on the image.

import heapq

def dijkstra_with_predecessors(graph, start):
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0
    predecessors = {node: None for node in graph}
    queue = [(0, start)]
    
    while queue:
        current_distance, current_node = heapq.heappop(queue)
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node
                heapq.heappush(queue, (distance, neighbor))
    
    return distances, predecessors

def backtrack_path(predecessors, start, end):
    path = []
    current_node = end
    while current_node != start:
        path.append(current_node)
        current_node = predecessors[current_node]
        if current_node is None:
            # No path exists between start and end
            return None
    path.append(start)  # optional: include start point in the path
    path.reverse()  # optional: reverse path to start->end
    return path

def draw_path_on_image(image, path, color_value):
    # Assuming a 3-channel color image and the path color value is a tuple (R, G, B)
    for position in path:
        image[position[0], position[1]] = color_value
    return image

# Now, let's load the image, calculate the gradients, and construct the graph.
image_path = 'Complete test/Small/Case12.bmp'
color_image = Image.open(image_path)
gray_image = color_image.convert('L')
image_array = np.array(gray_image)

# Calculate gradients using the sobel_filters function
Gx, Gy, G = sobel_filters(image_array)

# Construct the graph with the calculated gradient magnitudes
graph = construct_graph_with_gradients(image_array, G)

# Define start (anchor) and end (free) points for demonstration
start_point = (0, 0)  # Bottom left corner
end_point = (image_array.shape[0] - 1, image_array.shape[1] - 1)  # Top right corner

# Run Dijkstra's algorithm
distances, predecessors = dijkstra_with_predecessors(graph, start_point)

# Backtrack to find the shortest path from end to start
path = backtrack_path(predecessors, start_point, end_point)

# Now, let's draw the path on the original color image if the path exists
if path is not None:
    # Define the color value for the path, e.g., red
    path_color_value = (255, 0, 0)  # Red color for the path
    # Convert the color image to a NumPy array for drawing
    color_image_array = np.array(color_image)
    # Draw the path
    color_image_with_path = draw_path_on_image(color_image_array, path, path_color_value)
    # Convert the NumPy array back to an image
    output_image = Image.fromarray(color_image_with_path)
else:
    output_image = color_image  # No path found, return the original image

# Save the output image with the path
output_image_path = 'Complete test/Small/Case12.bmp'
output_image.save(output_image_path)
output_image_path


'Complete test/Small/Case12.bmp'

In [14]:
#tkinter
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

class InteractiveScissors:
    def __init__(self, root):
        self.root = root
        self.canvas = tk.Canvas(root, cursor="cross")
        self.canvas.pack(fill="both", expand=True)
        self.image_path = filedialog.askopenfilename()
        self.image = Image.open(self.image_path)
        self.photo = ImageTk.PhotoImage(self.image)
        self.canvas.create_image(0, 0, image=self.photo, anchor="nw")
        self.start_point = None
        self.end_point = None
        self.canvas.bind("<Button-1>", self.on_click)

    def on_click(self, event):
        if not self.start_point:
            self.start_point = (event.y, event.x)
            print(f"Start point set at {self.start_point}")
        else:
            self.end_point = (event.y, event.x)
            print(f"End point set at {self.end_point}")
            self.root.quit()  

root = tk.Tk()
app = InteractiveScissors(root)
root.mainloop()


Start point set at (110, 461)
End point set at (120, 469)


: 